Your Name: Kieran Fitzmaurice

Many probabilistic systems with discrete states can be simulated with random numbers. As an analogy for these systems, let's look at the more familiar 6-sided die.

If you roll a single 6-sided die, the result is a uniform distribution of values from 1 to 6. If you roll two dice and sum them, you get values from 2 to 12, with 7 being the most likely, etc. In general, the behavior of any number of dice is well described by the [binormal distribution](https://en.wikipedia.org/wiki/Binomial_distribution).

But, what happens if we modify these rules? Suppose that if a die rolls 6, we get to re-roll it and add the second result to the first. If we roll a second 6, we add it and then roll the die again, and so on. For example, we could roll 6+6+6+2 for a total of 20 on a single die. In principle, there is no limit to how high a number we could roll using this scheme (although large values become increasingly unlikely). Let's call this an "opened-ended die" If we repeat this procedure with multiple open-ended dice, we should get something resembling a distorted binomial distribution.

Write a program that "rolls" $N$ open-ended dice and sums the total value rolled, with this process repeated $M$ times, where $N$ and $M$ are both selected using widgets from `ipywidgets`. Let $N$ range from 1 to 20 dice, and let $M$ range from 100 to 10000 trials (in steps of 100).

Plot the resulting distribution as a bar chart, with the y-axis of your chart showing the **probability** that the sum has a given value. For example, if a value has a probability of 0.2, that means means that there is a 20% chance of rolling that value. Put another way, changing the value of $M$ should not change the vertical scale of your chart. It should only affect the amount of noise.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wgt

N_slider = wgt.IntSlider(description = "N = ", min=1, max=20, step=1, value=1)
M_slider = wgt.IntSlider(description = "M = ", min=100, max = 10000, step=1, value = 100)

@wgt.interact(N = N_slider, M = M_slider)
def prob_dist(N,M):
    
    Sums = np.zeros(M)
    
    for i in range(0,M):
        S = 0;
        n = N;
        
        while(n > 0):
            dice = np.random.randint(1,7,n)
            S = S + np.sum(dice)
            n = np.sum(dice == 6)
            
        Sums[i] = S
        
    hist, bin_edges = np.histogram(Sums, bins = "auto", density=True)
    plt.bar(bin_edges[:-1], hist, align = 'edge', width = bin_edges[1] - bin_edges[0])
    plt.xlabel("Sum of Dice")
    plt.ylabel("Probability Density")
    
    plt.show()
    


As it turns out, a normal (Gaussian) distribution is a decent fit for multiple open-ended dice, while the binomial distribution is rather poor. Modify your program to fit the normal distribution to your dice rolls using `scipy.curve_fit`. The normal distribution is given by
$$P(x) =  \frac{1}{\sqrt{2\pi\sigma^2}}\exp\Bigg(\frac{(x-\mu)^2}{2\sigma^2}\Bigg)\,,$$
where $\mu$ is the mean and $\sigma$ is the standard deviation.

Plot your best fit distribution on top of your bar chart, and print out the best-fit value of $\mu$ (the average total you'd expect to roll given $N$ open-ended dice). Your plot should still be interactive like it was in the first part.

Note: You may need to supply an initial guess for $\mu$ and $\sigma$ or else `curve_fit` may not converge.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wgt
from scipy.optimize import curve_fit

N_slider = wgt.IntSlider(description = "N = ", min=1, max=20, step=1, value=20)
M_slider = wgt.IntSlider(description = "M = ", min=100, max = 10000, step=1, value = 10000)

@wgt.interact(N = N_slider, M = M_slider)
def prob_dist(N,M):
    
    Sums = np.zeros(M)
    
    for i in range(0,M):
        S = 0;
        n = N;
        
        while(n > 0):
            dice = np.random.randint(1,7,n)
            S = S + np.sum(dice)
            n = np.sum(dice == 6)
            
        Sums[i] = S
        
    hist, bin_edges = np.histogram(Sums, bins = "auto", density=True)
    bin_centers = (bin_edges[1:] + bin_edges[:-1]) / 2
    
    
    meu_guess = np.mean(bin_centers)
    sigma_guess = np.std(bin_centers)
    
    def fit(x,sigma,meu):
        f = 1/(np.sqrt(2*np.pi*sigma**2))*np.exp(-1*((x-meu)**2/(2*sigma**2)))
        return(f)
    
    popt, pcov = curve_fit(fit, bin_centers, hist, p0=(meu_guess,sigma_guess))
    sigma_fit, meu_fit = popt
    
    plt.bar(bin_edges[:-1], hist, align = 'edge', width = bin_edges[1] - bin_edges[0])
    
    min_x,max_x = plt.gca().get_xlim()
    lotsx = np.linspace(min_x,max_x,300)
    lotsy = fit(lotsx,sigma_fit,meu_fit)
    
    plt.plot(lotsx,lotsy,'r-')
    plt.axis(xmin = min_x,xmax = max_x)
    plt.title("Dice Game")
    plt.xlabel("Sum of dice")
    plt.ylabel("Probability Density")
    plt.show()
    
    print("Sigma = %lf, Meu = %lf" % (sigma_fit, meu_fit))